# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [294]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DF_practice").getOrCreate()

spark

In [295]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
cores

1

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [296]:
path = "../data/Rep_vs_Dem_tweets.csv"
df = spark.read.csv(path,inferSchema=True,header=True)


In [297]:
df.count()

92489

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [298]:
df.limit(5).toPandas()

Party         Handle  \
0                            Democrat  RepDarrenSoto   
1                            Democrat  RepDarrenSoto   
2                            Democrat  RepDarrenSoto   
3  Congress has allocated about $18…"           None   
4                            Democrat  RepDarrenSoto   

                                               Tweet  
0  Today, Senate Dems vote to #SaveTheInternet. P...  
1  RT @WinterHavenSun: Winter Haven resident / Al...  
2  RT @NBCLatino: .@RepDarrenSoto noted that Hurr...  
3                                               None  
4  RT @NALCABPolicy: Meeting with @RepDarrenSoto ...

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [300]:
df.select(df.Tweet).show(5,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|null                                                                                                                                        |

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [301]:
df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [303]:
from pyspark.sql.functions import * #regexp_extract



#method1
df.select("*").filter(df.Tweet.like("%@LatinoLeader%")).show(5,False)



+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy: Meeting with @RepDarrenSoto . Thanks for taking the time to meet with @LatinoLeader ED Marucci Guzman. #NALCABPolicy2018.…|
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+



In [304]:
#method2... IDK


df.filter(array_contains(split("Tweet"," "),"@LatinoLeader") ==True).select("*").show()



+--------+-------------+--------------------+
|   Party|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------+-------------+--------------------+



In [305]:
#method3 

latino = df.withColumn("Latino_Mentions",regexp_extract(df.Tweet,"(.)(@LatinoLeader)(.)",2))
latino.limit(5).toPandas()

Party         Handle  \
0                            Democrat  RepDarrenSoto   
1                            Democrat  RepDarrenSoto   
2                            Democrat  RepDarrenSoto   
3  Congress has allocated about $18…"           None   
4                            Democrat  RepDarrenSoto   

                                               Tweet Latino_Mentions  
0  Today, Senate Dems vote to #SaveTheInternet. P...                  
1  RT @WinterHavenSun: Winter Haven resident / Al...                  
2  RT @NBCLatino: .@RepDarrenSoto noted that Hurr...                  
3                                               None            None  
4  RT @NALCABPolicy: Meeting with @RepDarrenSoto ...   @LatinoLeader

## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [311]:
counts = df.groupBy("Party").count()
counts.orderBy(desc("count")).show(6,False)
#counts.orderBy(counts["count"].desc()).show(6,False)

+-----------------------+-----+
|Party                  |count|
+-----------------------+-----+
|Republican             |44392|
|Democrat               |42068|
|That’s…"               |28   |
|https://t.co/oc6JNAF5K5|22   |
|Now                    |17   |
|Today                  |13   |
+-----------------------+-----+
only showing top 6 rows

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
|               Today|   13|
|         https://t…"|   12|
|              http…"|   12|
|                 h…"|   12|
|  ❌ Terminated #DACA|   11|
|   ❌ Passed #TaxScam|   11|
|        #mepolitics"|   11|
|               ❌ E…"|   11|
|❌ Abandoned Hispa...|   11|
|      https://t.co…"|    9|
|https://t.co/8htz...|    9|
|💻 Website: https...|    9|
|This. Has. To. St...|    8|
|Semper Fidelis. h...|    7|
|                 

In [315]:
from pyspark.sql.functions import when

clean = df.withColumn("Party", (when(df.Party=="Democrat","Democrat").when(df.Party=="Republican","Republican").otherwise("Other")))
counts = clean.groupBy("Party").count()
counts.orderBy(desc("count")).show()


+----------+-----+
|     Party|count|
+----------+-----+
|Republican|44392|
|  Democrat|42068|
|     Other| 6029|
+----------+-----+



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [51]:
from pyspark.sql.functions import col, regexp_replace
df.select(df.Tweet,regexp_replace(df.Tweet,r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b',"")).show(1,False)


+-------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                      |regexp_replace(Tweet, (https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b, )                                                |
+-------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrali

## 4. Remove any leading or trailing white space in the tweet column

In [55]:
from pyspark.sql.functions import trim,ltrim,rtrim
df.select(trim(df.Tweet).alias("Tweet")).show()

+--------------------+
|               Tweet|
+--------------------+
|Today, Senate Dem...|
|RT @WinterHavenSu...|
|RT @NBCLatino: .@...|
|                null|
|RT @NALCABPolicy:...|
|RT @Vegalteno: Hu...|
|RT @EmgageActionF...|
|Hurricane Maria l...|
|RT @Tharryry: I a...|
|RT @HispanicCaucu...|
|RT @RepStephMurph...|
|RT @AllSaints_FL:...|
|.@realDonaldTrump...|
|Thank you to my m...|
|We paid our respe...|
|                null|
|RT @WinterHavenSu...|
|Meet 12 incredibl...|
|RT @wildlifeactio...|
|RT @CHeathWFTV: K...|
+--------------------+
only showing top 20 rows



## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [57]:


df.withColumnRenamed("Party","Dem_Rep").show()

+--------------------+-------------+--------------------+
|             Dem_Rep|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|            Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|            Democrat|RepDarrenSoto|RT @EmgageActionF...|
|            Democrat|RepDarrenSoto|Hurricane Maria l...|
|            Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|            Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|            Democrat|RepDarrenSoto|RT @RepStephMurph...|
|            Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|            Democrat|RepDarrenSoto|.@realDonaldTrump...|
|            Democrat|RepDarrenSoto|Thank you to my m...|
|            D

## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [63]:
from pyspark.sql.functions import concat_ws

df.select("Party","Handle",concat_ws(" ","Party","Handle").alias("concon")).show()

#or

df.select("Party","Handle",concat_ws(" ",df.Party,df.Handle)).show()

+--------------------+-------------+--------------------+
|               Party|       Handle|              concon|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|Congress has allo...|         null|Congress has allo...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            Democrat|RepDarrenSoto|Democrat RepDarre...|
|            D

## Challenge Question

Let's imagine that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

### Regular expression!

- the " . " maches any character other than a new line
- " | " is similar to "or"
- \w+  means followed by any word

- " * " is used to match the preceding character **zero or more times**
- " ? " will match the preceding character **zero or one, but no more.**
- " $ " is used to match the ending position in a string



In [319]:
pattern = '(.|'')(#)(\w+)'
split_pattern = r'.*?({pattern})'.format(pattern=pattern)
end_pattern = r'(.*{pattern}).*?$'.format(pattern=pattern)

print(split_pattern,end_pattern)

.*?((.|)(#)(\w+)) (.*(.|)(#)(\w+)).*?$


In [82]:
from pyspark.sql.functions import regexp_extract
# df.filter(df.Tweet.like("%#%")).select(regexp_extract(df.Tweet,r"(#[a-zA-Z0-9]*)",0).alias("HashTag")).show()

df.filter(df.Tweet.like("%#%")).select(regexp_extract(df.Tweet,r"(#\w*)",0).alias("HashTag")).show()

+--------------------+
|             HashTag|
+--------------------+
|    #SaveTheInternet|
|   #NALCABPolicy2018|
|      #NetNeutrality|
|            #Orlando|
|             #HFAnow|
|#classroomwhereit...|
|               #FACT|
|        #ABetterDeal|
|            #Science|
|         #2020Census|
|          #ghostguns|
|              #APAHM|
|        #Insulin4all|
|      #EmgagetheHill|
|      #Emgagethehill|
|                #TPS|
|     #HurricaneMaria|
|           #vamos4pr|
|          #DemsAtWor|
|         #DemsAtWork|
+--------------------+
only showing top 20 rows



# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients' first and last names, date of birth, and the dates of their first 3 visits. 

In [322]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [323]:
from pyspark.sql.types import * 

df = df.withColumn('dob',df['dob'].cast(DateType()))\
        .withColumn("visit1",df.visit1.cast(DateType()))\
        .withColumn("visit2",df.visit2.cast(DateType()))\
        .withColumn("visit3",df.visit3.cast(DateType()))

df.toPandas()

first_name  last_name         dob      visit1      visit2      visit3
0   Mohammed     Alfasy  1987-04-08  2016-01-07  2017-02-03  2018-03-02
1      Marcy  Wellmaker  1986-04-08  2015-01-07  2017-01-03  2018-01-02
2      Ginny     Ginger  1986-07-10  2014-08-07  2015-02-03  2016-03-02
3      Vijay   Doberson  1988-05-02  2016-01-07  2018-02-03  2018-03-02
4      Orhan    Gelicek  1987-05-11  2016-05-07  2017-01-03  2018-09-02
5      Sarah      Jones  1956-07-06  2016-04-07  2017-08-03  2018-10-02
6       John    Johnson  2017-10-12  2018-01-02  2018-10-03  2018-03-02

## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [324]:
from pyspark.sql.functions import *

#In the form of years and days.
df.select((df.visit1-df.visit2)\
          .alias("between first and second"),(df.visit2-df.visit3)\
          .alias("between second and third")).show()


+------------------------+------------------------+
|between first and second|between second and third|
+------------------------+------------------------+
|       -1 years -27 days|       -1 years -27 days|
|    -1 years -11 mont...|     -11 months -30 days|
|      -5 months -27 days|       -1 years -27 days|
|       -2 years -27 days|                -27 days|
|      -7 months -27 days|    -1 years -7 month...|
|    -1 years -3 month...|    -1 years -1 month...|
|       -9 months -1 days|         7 months 1 days|
+------------------------+------------------------+



In [331]:
from pyspark.sql.functions import datediff

#numbers in days

diff1 = df.select(datediff(df.visit2,df.visit1).alias("diff"))
diff2= df.select(datediff(df.visit3,df.visit2).alias("diff"))

#Append the two dataframes together

diff_combo = diff1.union(diff2)
diff_combo.show()

+----+
|diff|
+----+
| 393|
| 727|
| 180|
| 758|
| 241|
| 483|
| 274|
| 392|
| 364|
| 393|
|  27|
| 607|
| 425|
|-215|
+----+



In [332]:
df.select(avg(regexp_replace((to_timestamp(df.visit2)-to_timestamp(df.visit1)).cast(StringType())," hours","").cast(IntegerType()))).show()

+-------------------------------------------------------------------------------------------------------------+
|avg(CAST(regexp_replace(CAST((to_timestamp(`visit2`) - to_timestamp(`visit1`)) AS STRING),  hours, ) AS INT))|
+-------------------------------------------------------------------------------------------------------------+
|                                                                                           10477.714285714286|
+-------------------------------------------------------------------------------------------------------------+



## 8. Can you calculate the age of each patient?

In [336]:
from pyspark.sql.functions import format_number
#1
df.select(format_number(datediff(current_date(),df.dob)/365,1)).show()


+-------------------------------------------------------+
|format_number((datediff(current_date(), dob) / 365), 1)|
+-------------------------------------------------------+
|                                                   34.4|
|                                                   35.4|
|                                                   35.2|
|                                                   33.4|
|                                                   34.4|
|                                                   65.2|
|                                                    3.9|
+-------------------------------------------------------+



In [326]:
#2
df.select(concat_ws(" ",df.first_name, df.last_name)\
          .alias("name"),current_date() - df.dob).show()

+---------------+----------------------+
|           name|(current_date() - dob)|
+---------------+----------------------+
|Mohammed Alfasy|  34 years 5 months...|
|Marcy Wellmaker|  35 years 5 months...|
|   Ginny Ginger|  35 years 1 months...|
| Vijay Doberson|  33 years 4 months...|
|  Orhan Gelicek|  34 years 3 months...|
|    Sarah Jones|  65 years 2 months...|
|   John Johnson|  3 years 10 months...|
+---------------+----------------------+



## 9. Can you extract the month from the first visit column and call it "Month"?

In [165]:
#1 regular expression
df.select(regexp_extract(df.visit1,"([0-9]*)-([0-9]*)-([0-9]*)",2)\
          .alias("Month")).show()

+-----+
|Month|
+-----+
|   01|
|   01|
|   08|
|   01|
|   05|
|   04|
|   01|
+-----+



In [337]:
from pyspark.sql.functions import month

#2month function
df.select(month(df['visit1']).alias("Month")).show()

+-----+
|Month|
+-----+
|    1|
|    1|
|    8|
|    1|
|    5|
|    4|
|    1|
+-----+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [340]:
path="../data/supermarket_sales.csv"
df = spark.read.csv(path,inferSchema=True,header=True)


## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [341]:
df.show(4,False)

+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|Invoice ID |Branch|City     |Customer type|Gender|Product line          |Unit price|Quantity|Tax 5% |Total   |Date     |Time |Payment    |cogs  |gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+----------------------+----------+--------+-------+--------+---------+-----+-----------+------+-----------------------+------------+------+
|750-67-8428|A     |Yangon   |Member       |Female|Health and beauty     |74.69     |7       |26.1415|548.9715|1/5/2019 |13:08|Ewallet    |522.83|4.761904762            |26.1415     |9.1   |
|226-31-3081|C     |Naypyitaw|Normal       |Female|Electronic accessories|15.28     |5       |3.82   |80.22   |3/8/2019 |10:29|Cash       |76.4  |4.761904762            |3.82        |9.6   |
|631-41-3108|A     |Yangon   |Normal       |M

In [342]:
df.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [345]:
print("Note that this method gives all null values")
df = df.withColumn("Formatted Date", df["Date"].cast(DateType()))
df = df.select("Date","Formatted Date")
print(df.limit(6).toPandas())

Note that this method gives all null values
        Date Formatted Date
0   1/5/2019           None
1   3/8/2019           None
2   3/3/2019           None
3  1/27/2019           None
4   2/8/2019           None
5  3/25/2019           None


In [347]:
print(" ")
print("This result gives the wrong results (notice that all months are january)")
df.select('Date',to_date(df.Date, 'm/d/yyyy').alias('Dateformatted'),month(to_date(df.Date, 'm/d/yyyy')).alias('Month'),).show(5)


 
This result gives the wrong results (notice that all months are january)
+---------+-------------+-----+
|     Date|Dateformatted|Month|
+---------+-------------+-----+
| 1/5/2019|   2019-01-05|    1|
| 3/8/2019|   2019-01-08|    1|
| 3/3/2019|   2019-01-03|    1|
|1/27/2019|   2019-01-27|    1|
| 2/8/2019|   2019-01-08|    1|
+---------+-------------+-----+
only showing top 5 rows



In [353]:
print("But if we capitalize the mm part in the format, we get the correct results!")
df.select('Date',to_date(df.Date, 'M/d/yyyy').alias('Dateformatted'),month(to_date(df.Date, 'M/d/yyyy')).alias('Month'),).toPandas()

But if we capitalize the mm part in the format, we get the correct results!


Date Dateformatted  Month
0     1/5/2019    2019-01-05      1
1     3/8/2019    2019-03-08      3
2     3/3/2019    2019-03-03      3
3    1/27/2019    2019-01-27      1
4     2/8/2019    2019-02-08      2
..         ...           ...    ...
995  1/29/2019    2019-01-29      1
996   3/2/2019    2019-03-02      3
997   2/9/2019    2019-02-09      2
998  2/22/2019    2019-02-22      2
999  2/18/2019    2019-02-18      2

[1000 rows x 3 columns]

In [198]:
#UDF! 

from datetime import datetime

date_udf = udf(lambda x: datetime.strptime(x, "%m/%d/%Y"), DateType())

df.select("Date",date_udf("Date").alias("converted time")).toPandas() #this seems right!


#It is noteworthy, that in order to parse through a 2-digit year, 
#e.g. '90' rather than '1990', a %y is required instead of a %Y.

Date converted time
0     1/5/2019     2019-01-05
1     3/8/2019     2019-03-08
2     3/3/2019     2019-03-03
3    1/27/2019     2019-01-27
4     2/8/2019     2019-02-08
..         ...            ...
995  1/29/2019     2019-01-29
996   3/2/2019     2019-03-02
997   2/9/2019     2019-02-09
998  2/22/2019     2019-02-22
999  2/18/2019     2019-02-18

[1000 rows x 2 columns]

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [358]:
df = df.select('Date',split(df.Date, '/')[0].alias('Month'))


df.show(5)


+---------+-----+
|     Date|Month|
+---------+-----+
| 1/5/2019|    1|
| 3/8/2019|    3|
| 3/3/2019|    3|
|1/27/2019|    1|
| 2/8/2019|    2|
+---------+-----+
only showing top 5 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [359]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.  '), \
          (4,'Solid but I would have liked to see more of the love story  '), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+----------------------------------------------------------------------------------------+
|rating|review_txt                                                                              |
+------+----------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                           |
|4     |Pretty good, but I would have liked to seen better special effects                      |
|3     |So so. Casting could have been improved                                                 |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.  |
|4     |Solid but I would have liked to see more of the love story                              |
|5     |THE BOMB!!!!!!!                                                                         |
+------+----------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [361]:
#1 remove all punctuation
reviews.select(regexp_replace(reviews.review_txt,r"[./,!]","").alias("renewed")).show(6,False)

+-------------------------------------------------------------------------------------+
|renewed                                                                              |
+-------------------------------------------------------------------------------------+
|Epic This is the best movie I have EVER seen                                         |
|Pretty good but I would have liked to seen better special effects                    |
|So so Casting could have been improved                                               |
|The most EPIC movie of the year Casting was awesome Special effects were so intense  |
|Solid but I would have liked to see more of the love story                           |
|THE BOMB                                                                             |
+-------------------------------------------------------------------------------------+



In [245]:
#2 lower case everything
#2-1
udf_lower = udf(lambda x: x.lower(),StringType())
reviews.select(udf_lower(reviews.review_txt)).show()

+--------------------+
|<lambda>(review_txt)|
+--------------------+
|epic. this is the...|
|pretty good, but ...|
|so so. casting co...|
|the most epic mov...|
|solid but i would...|
|     the bomb!!!!!!!|
+--------------------+



In [246]:
#2-2
reviews.select(lower(reviews.review_txt)).show()

+--------------------+
|   lower(review_txt)|
+--------------------+
|epic. this is the...|
|pretty good, but ...|
|so so. casting co...|
|the most epic mov...|
|solid but i would...|
|     the bomb!!!!!!!|
+--------------------+



In [259]:
#3 Remove white space (trim)

reviews.select(trim(reviews.review_txt).alias("Trimmed"),length(trim(reviews.review_txt)).alias("len-trimmed"),length(reviews.review_txt).alias("len-untrimmed")).toPandas()

Trimmed  len-trimmed  \
0      Epic. This is the best movie I have EVER seen           45   
1  Pretty good, but I would have liked to seen be...           66   
2            So so. Casting could have been improved           39   
3  The most EPIC movie of the year! Casting was a...           86   
4  Solid but I would have liked to see more of th...           58   
5                                    THE BOMB!!!!!!!           15   

   len-untrimmed  
0             45  
1             66  
2             39  
3             88  
4             60  
5             15

In [265]:
#Then finally, split the string
reviews.select(split(regexp_replace(lower(trim(reviews.review_txt)),r"[./,!]","")," ").alias("Final")).show(20,False)


+---------------------------------------------------------------------------------------------------+
|Final                                                                                              |
+---------------------------------------------------------------------------------------------------+
|[epic, this, is, the, best, movie, i, have, ever, seen]                                            |
|[pretty, good, but, i, would, have, liked, to, seen, better, special, effects]                     |
|[so, so, casting, could, have, been, improved]                                                     |
|[the, most, epic, movie, of, the, year, casting, was, awesome, special, effects, were, so, intense]|
|[solid, but, i, would, have, liked, to, see, more, of, the, love, story]                           |
|[the, bomb]                                                                                        |
+---------------------------------------------------------------------------------

## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [362]:
reviews.select(split(regexp_replace(lower(trim(reviews.review_txt)),r"[./,!]","")," ").alias("Things to check"),array_contains(split(regexp_replace(lower(trim(reviews.review_txt)),r"[./,!]","")," "),"epic").alias("Final")).toPandas()


Things to check  Final
0  [epic, this, is, the, best, movie, i, have, ev...   True
1  [pretty, good, but, i, would, have, liked, to,...  False
2     [so, so, casting, could, have, been, improved]  False
3  [the, most, epic, movie, of, the, year, castin...   True
4  [solid, but, i, would, have, liked, to, see, m...  False
5                                        [the, bomb]  False

### That's it! Great Job!